# Preliminary Data Exploration
This notebook takes a look at the [data provided](https://github.com/HACC2020/data) by the UH Occupancy challenge for HACC 2020. 

In [35]:
import pandas as pd
import numpy as np
from pprint import pprint
from datetime import datetime

In [36]:
df = pd.read_csv('data-main/uh_occupancy/2020-0824_600am-859am_devices_1598295707.csv')
df2 = pd.read_csv('data-main/uh_occupancy/2020-0824_900am-1159am_devices_1598306516.csv')

In [37]:
pprint(df)
pprint(df2)

                     Device  Max Clients  Unique Clients     Building
0       Wainani H 362B Temp            2               2      Wainani
1         Frear D 507C Temp            5               9        Frear
2         Frear D 207C Temp            4              10        Frear
3         Frear D 707C Temp            4               5        Frear
4           Noelani E Store            1              10      Noelani
...                     ...          ...             ...          ...
3209    C-MORE 104 Outreach            0               4       C-MORE
3210            C-MORE 202A            0               0       C-MORE
3211             C-MORE 218            0               1       C-MORE
3212  Athletics 345 Hallway            4              12    Athletics
3213         CBA G103 Mauka            0               0  Shidler CBA

[3214 rows x 4 columns]
                     Device  Max Clients  Unique Clients     Building
0       Wainani H 362B Temp            2               2      Wai

In [38]:
(df['Unique Clients'] > df['Max Clients']).value_counts()
(df2['Unique Clients'] > df2['Max Clients']).value_counts()

True     2497
False     717
dtype: int64

In [77]:
import pandas as pd
import glob

path='data-main/uh_occupancy'

all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    
    filename = filename.split('/')
    filename = filename[-1:][0].split('_')
    filename.remove('devices')
    df2 = pd.DataFrame([filename], columns=['date', 'period', 'epoch'])
    
    df = pd.concat([df, df2], axis=1, join='inner')
    
    li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True)

In [81]:
pprint(frame['Building 37'].dropna())

31    Hamilton Library
Name: Building 37, dtype: object


In [41]:
test = 'data-main/uh_occupancy/2020-0824_1200pm-259pm_devices_1598317333.csv'

filename = test.split('/')
filename = filename[-1:][0].split('_')

filename.remove('devices')
print(filename)

['2020-0824', '1200pm-259pm', '1598317333.csv']


In [46]:
df3 = pd.DataFrame([filename], columns=['date', 'period', 'epoch'])

In [47]:
df3[['date','period','epoch']] = filename
print(df3)

        date        period           epoch
0  2020-0824  1200pm-259pm  1598317333.csv
